<a href="https://colab.research.google.com/github/MoralesCMaJo/Tarea-2--An-lisis-de-series-temporales/blob/main/10_The_GARCH_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the relevant packages


In [ ]:
!pip install arch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 10.7 MB/s eta 0:00:00


Importing the Data and Pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
# Ruta del archivo en Google Drive
ruta_drive = "/content/drive/My Drive/Index_2023.csv"
# Ruta de destino en Google Colab
ruta_colab = "/content/Index_2023.csv"
# Copia el archivo desde Google Drive a Google Colab
shutil.copy(ruta_drive, ruta_colab)

#Importación de datos y preprocesamiento
raw_csv_data = pd.read_csv("Index_2023.csv")
df_comp=raw_csv_data.copy()
df_comp.Date = pd.to_datetime(df_comp.Date, dayfirst = True)
df_comp.set_index("Date", inplace=True)
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

Mounted at /content/drive


In [ ]:
df_comp['market_value']=df_comp.ftse



In [ ]:
del df_comp['spx']
del df_comp['dax']
del df_comp['ftse']
del df_comp['nikkei']
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]
import warnings
warnings.filterwarnings("ignore")

The LLR Test

In [ ]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR, DF).round(3)
    return p

Creating Returns

In [ ]:
df['returns'] = df.market_value.pct_change(1)*100


The Simple GARCH Model

In [ ]:
model_garch_1_1 = arch_model(df.returns[1:], mean = "Constant", vol = "GARCH", p = 1, q = 1)
results_garch_1_1 = model_garch_1_1.fit(update_freq = 5)
results_garch_1_1.summary()

Iteration:      5,   Func. Count:     35,   Neg. LLF: 8364.108094821579
Iteration:     10,   Func. Count:     65,   Neg. LLF: 8357.41091025245
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8357.41091025245
            Iterations: 11
            Function evaluations: 69
            Gradient evaluations: 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8357.41
Distribution:                  Normal   AIC:                           16722.8
Method:            Maximum Likelihood   BIC:                           16749.8
                                        No. Observations:                 6199
Date:                Sat, Oct 14 2023   Df Residuals:                     6198
Time:                        06:11:33   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0431  1.048e-02      4.113  3.909e-05 [2.255e-02,6.361e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0132  3.437e-03      3.839  1.233e-04 [6.460e-03,1.993e-02]
alpha[1]       0.0890  1.149e-02      7.746  9.511e-15   [6.645e-02,  0.111]
beta[1]        0.9002  1.297e-02     69.401      0.000     [  0.875,  0.926]
============================================================================

Covariance estimator: robust
"""

Higher-Lag GARCH Models

In [ ]:
model_garch_1_2 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 2)
results_garch_1_2 = model_garch_1_2.fit(update_freq = 5)
results_garch_1_2.summary()

Iteration:      5,   Func. Count:     40,   Neg. LLF: 8368.276321909765
Iteration:     10,   Func. Count:     72,   Neg. LLF: 8357.413559632852
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8357.410910333732
            Iterations: 14
            Function evaluations: 95
            Gradient evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8357.41
Distribution:                  Normal   AIC:                           16724.8
Method:            Maximum Likelihood   BIC:                           16758.5
                                        No. Observations:                 6199
Date:                Sat, Oct 14 2023   Df Residuals:                     6198
Time:                        06:12:06   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0431  1.047e-02      4.116  3.854e-05 [2.257e-02,6.360e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0132  3.297e-03      4.003  6.265e-05 [6.735e-03,1.966e-02]
alpha[1]       0.0890  1.368e-02      6.501  7.976e-11   [6.214e-02,  0.116]
beta[1]        0.9002      0.206      4.374  1.218e-05     [  0.497,  1.304]
beta[2]    3.6238e-12      0.196  1.846e-11      1.000     [ -0.385,  0.385]
============================================================================

Covariance estimator: robust
"""

In [ ]:
model_garch_1_3 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 3)
results_garch_1_3 = model_garch_1_3.fit(update_freq = 5)
results_garch_1_3.summary()

Iteration:      5,   Func. Count:     47,   Neg. LLF: 8493.565054337927
Iteration:     10,   Func. Count:     87,   Neg. LLF: 8357.593961897674
Iteration:     15,   Func. Count:    122,   Neg. LLF: 8357.410910206818
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8357.410910206818
            Iterations: 15
            Function evaluations: 122
            Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8357.41
Distribution:                  Normal   AIC:                           16726.8
Method:            Maximum Likelihood   BIC:                           16767.2
                                        No. Observations:                 6199
Date:                Sat, Oct 14 2023   Df Residuals:                     6198
Time:                        06:12:26   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0431  2.143e-02      2.011  4.436e-02 [1.087e-03,8.508e-02]
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          0.0132  6.723e-02      0.196      0.844 [ -0.119,  0.145]
alpha[1]       0.0890      0.467      0.190      0.849 [ -0.827,  1.005]
beta[1]        0.9002     13.766  6.539e-02      0.948 [-26.081, 27.882]
beta[2]    7.5260e-11     20.991  3.585e-12      1.000 [-41.142, 41.142]
beta[3]        0.0000      7.745      0.000      1.000 [-15.180, 15.180]
========================================================================

Covariance estimator: robust
"""

In [ ]:
model_garch_2_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 2, q = 1)
results_garch_2_1 = model_garch_2_1.fit(update_freq = 5)
results_garch_2_1.summary()

Iteration:      5,   Func. Count:     41,   Neg. LLF: 8374.272287037737
Iteration:     10,   Func. Count:     76,   Neg. LLF: 8358.744149762155
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8356.28400637422
            Iterations: 14
            Function evaluations: 100
            Gradient evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8356.28
Distribution:                  Normal   AIC:                           16722.6
Method:            Maximum Likelihood   BIC:                           16756.2
                                        No. Observations:                 6199
Date:                Sat, Oct 14 2023   Df Residuals:                     6198
Time:                        06:12:45   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0428  1.051e-02      4.075  4.605e-05 [2.224e-02,6.345e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0150  4.700e-03      3.197  1.387e-03  [5.816e-03,2.424e-02]
alpha[1]       0.0710  1.720e-02      4.129  3.645e-05    [3.732e-02,  0.105]
alpha[2]       0.0259  2.449e-02      1.057      0.290 [-2.210e-02,7.389e-02]
beta[1]        0.8908  1.928e-02     46.212      0.000      [  0.853,  0.929]
=============================================================================

Covariance estimator: robust
"""

In [ ]:
model_garch_3_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 3, q = 1)
results_garch_3_1 = model_garch_3_1.fit(update_freq = 5)
results_garch_3_1.summary()

Iteration:      5,   Func. Count:     45,   Neg. LLF: 15715.489255304889
Iteration:     10,   Func. Count:     85,   Neg. LLF: 8356.314622239566
Iteration:     15,   Func. Count:    119,   Neg. LLF: 8356.284006323696
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8356.284006321966
            Iterations: 15
            Function evaluations: 119
            Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -8356.28
Distribution:                  Normal   AIC:                           16724.6
Method:            Maximum Likelihood   BIC:                           16765.0
                                        No. Observations:                 6199
Date:                Sat, Oct 14 2023   Df Residuals:                     6198
Time:                        06:13:04   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0428  1.055e-02      4.060  4.909e-05 [2.216e-02,6.353e-02]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0150  5.936e-03      2.532  1.135e-02  [3.394e-03,2.666e-02]
alpha[1]       0.0710  1.720e-02      4.129  3.637e-05    [3.732e-02,  0.105]
alpha[2]       0.0259  2.463e-02      1.051      0.293 [-2.238e-02,7.415e-02]
alpha[3]       0.0000  2.632e-02      0.000      1.000 [-5.159e-02,5.159e-02]
beta[1]        0.8908  2.687e-02     33.145 6.764e-241      [  0.838,  0.943]
=============================================================================

Covariance estimator: robust
"""